In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ovrlpy

import sys
sys.path.extend(['../../scripts','../../scripts/xenium'])
import readwrite
cfg = readwrite.config()

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [ ]:
path = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/breast/1256/1256/normalised_results/outs/transcripts.parquet')
coordinate_df = pd.read_parquet(path).query("is_gene & (qv >= 20)") \
    .rename(
        columns={
            "x_location": "x",
            "y_location": "y",
            "z_location": "z",
            "feature_name": "gene",
        }) \
    .query("is_gene & (qv >= 20)")  # remove dummy molecules

coordinate_df["gene"] = coordinate_df["gene"].astype("category")

# coordinate_df[::1000].plot.scatter(x="x", y="y", s=1)
# coordinate_df = coordinate_df.copy()  # make a copy to avoid SettingWithCopyWarning

signal_integrity, signal_strength, visualizer = ovrlpy.run(
    df=coordinate_df, cell_diameter=10, n_expected_celltypes=20
)

fig, ax = ovrlpy.plot_signal_integrity(
    signal_integrity, signal_strength, signal_threshold=3
)

doublet_df = ovrlpy.detect_doublets(
    signal_integrity, signal_strength, minimum_signal_strength=3, integrity_sigma=2
)

_ = plt.scatter(
    doublet_df["x"],
    doublet_df["y"],
    c=doublet_df["integrity"],
    s=1,
    cmap="viridis",
    vmin=0,
    vmax=1,
)
plt.gca().set_aspect("equal")
plt.colorbar()


window_size = 60
doublet_case = 0

x, y = doublet_df.loc[doublet_case, ["x", "y"]]

ovrlpy.plot_region_of_interest(
    x,
    y,
    coordinate_df,
    visualizer,
    signal_integrity,
    signal_strength,
    window_size=window_size,
)


# visualizer.save(result_folder / "Xenium-text-visualizer.vis")

# vis = ovrlpy.load_visualizer(result_folder / "Xenium-text-visualizer.vis")

plt.figure(figsize=(20, 5))

ax = plt.subplot(111, projection="3d")

for i in range(-2, 3):
    subset = coordinate_df[(coordinate_df.z - coordinate_df.z_delim).between(i, i + 1)]

    ax.scatter(
        subset.x[::100],
        subset.y[::100],
        np.zeros(1 + (len(subset) // 100)) + i,
        s=1,
        alpha=0.1,
    )

Running vertical adjustment
Creating gene expression embeddings for visualization:
Analyzing in 3d mode:
determining pseudocells:
found 11537 pseudocells
sampling expression:


DMKN:  28%|██▊       | 77/280 [01:13<02:30,  1.35it/s]    

In [ ]:
def transcripts_to_count_matrix(transcripts,cell_column='cell_id',feature_column='feature_name', qv_treshold=20):
    transcripts = transcripts.query(f"(qv >= {qv_treshold}) & {cell_column} != 'UNASSIGNED'")
    cm = transcripts.pivot_table(index=cell_column, columns=feature_column, aggfunc='size', fill_value=0)
    return cm

integrity_filtering_threshold = 0.5

 # x and y are transposed in ovrlpy output. sanity check x and y must be transposed
assert (coordinate_df.y_pixel.max()+1) == signal_integrity.shape[0]
assert (coordinate_df.x_pixel.max()+1) == signal_integrity.shape[1]

coordinate_df['signal_integrity'] = signal_integrity[coordinate_df.y_pixel,coordinate_df.x_pixel]
coordinate_df['signal_strength'] = signal_strength[coordinate_df.y_pixel,coordinate_df.x_pixel]

coordinate_df_filtered = coordinate_df[coordinate_df.signal_integrity > integrity_filtering_threshold]
cell_mean_integrity = coordinate_df.query("cell_id != 'UNASSIGNED'").groupby('cell_id')['signal_integrity'].mean()
cell_is_singlet = cell_mean_integrity > integrity_filtering_threshold

count_matrix_filtered = transcripts_to_count_matrix(coordinate_df_filtered,feature_column='gene')

/tmp/ipykernel_947089/2333320809.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  cm = transcripts.pivot_table(index=cell_column, columns=feature_column, aggfunc='size', fill_value=0)


In [144]:
cell_mean_integrity.to_frame()

,signal_integrity
cell_id,
aaaabihh-1,0.228764
aaabddfm-1,0.160387
aaabeakn-1,0.657175
aaachkkh-1,0.763619
aaadbefk-1,0.703907
...,...
oimeelcd-1,0.000000
oimefhce-1,0.000000
oimefmmc-1,0.000000


,gene
gene,
ABCC11,ABCC11
ACTA2,ACTA2
ACTG2,ACTG2
ADAM9,ADAM9
ADGRE5,ADGRE5
...,...
VWF,VWF
WARS,WARS
ZEB1,ZEB1
